In [92]:
import tensorflow as tf
import pandas as pd
import numpy as np #numpy is the numerical computing package in python
import datetime #we'll use dates to label our logs

In [93]:

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from sklearn.preprocessing import StandardScaler

def make_tarnet(input_dim, reg_l2):
    '''
    The first argument is the column dimension of our data.
    It needs to be specified because the functional API creates a static computational graph
    The second argument is the strength of regularization we'll apply to the output layers
    '''
    x = Input(shape=(input_dim,), name='input')

    # REPRESENTATION
    #in TF2/Keras it is idiomatic to instantiate a layer and pass its inputs on the same line unless the layer will be reused
    #Note that we apply no regularization to the representation layers 
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)
    phi = Dense(units=200, activation='elu', kernel_initializer='RandomNormal',name='phi_3')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)

    # second layer
    y0_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=100, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    #a convenience "layer" that concatenates arrays as columns in a matrix
    concat_pred = Concatenate(1)([y0_predictions, y1_predictions])
    #the declarations above have specified the computational graph of our network, now we instantiate it
    model = Model(inputs=x, outputs=concat_pred)

    return model


In [94]:
 tarnet_model=make_tarnet(25,.01)

#print(tarnet_model.summary())
#tf.keras.utils.plot_model(tarnet_model, show_shapes=True, show_layer_names=True, to_file='tarnet.png')

#from IPython.display import Image # this just Jupyter notebook stuff
#Image(retina=True, filename='tarnet.png')
     

In [95]:
# every loss function in TF2 takes 2 arguments, a vector of true values and a vector predictions
def regression_loss(concat_true, concat_pred):
    #computes a standard MSE loss for TARNet
    y_true = concat_true[:, 0] #get individual vectors
    t_true = concat_true[:, 1]

    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]

    #Each head outputs a prediction for both potential outcomes
    #We use t_true as a switch to only calculate the factual loss
    loss0 = tf.reduce_sum((1. - t_true) * tf.square(y_true - y0_pred))
    loss1 = tf.reduce_sum(t_true * tf.square(y_true - y1_pred))
    #note Shi uses tf.reduce_sum for her losses instead of tf.reduce_mean.
    #They should be equivalent but it's possible that having larger gradients accelerates convergence.
    #You can always try changing it!
    return loss0 + loss1
     


In [96]:
#!wget -nc http://www.fredjo.com/files/ihdp_npci_1-100.train.npz
#!wget -nc http://www.fredjo.com/files/ihdp_npci_1-100.test.npz

In [97]:
def load_IHDP_data(training_data,testing_data,i=0):
    with open(training_data,'rb') as trf, open(testing_data,'rb') as tef:
        train_data=np.load(trf); test_data=np.load(tef)
        y=np.concatenate(   (train_data['yf'][:,i],   test_data['yf'][:,i])).astype('float32') #most GPUs only compute 32-bit floats
        t=np.concatenate(   (train_data['t'][:,i],    test_data['t'][:,i])).astype('float32')
        x=np.concatenate(   (train_data['x'][:,:,i],  test_data['x'][:,:,i]),axis=0).astype('float32')
        mu_0=np.concatenate((train_data['mu0'][:,i],  test_data['mu0'][:,i])).astype('float32')
        mu_1=np.concatenate((train_data['mu1'][:,i],  test_data['mu1'][:,i])).astype('float32')
 
        data={'x':x,'t':t,'y':y,'t':t,'mu_0':mu_0,'mu_1':mu_1}
        data['t']=data['t'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension 
        data['y']=data['y'].reshape(-1,1)
        
        #rescaling y between 0 and 1 often makes training of DL regressors easier
        data['y_scaler'] = StandardScaler().fit(data['y'])
        data['ys'] = data['y_scaler'].transform(data['y'])
 
    return data


def load_IHDP_data_n(training_data,testing_data,i,dt):
    
    if(dt=='train'):
        
        with open(training_data,'rb') as trf:
            train_data=np.load(trf)
            y=train_data['yf'][:,i].astype('float32') #most GPUs only compute 32-bit floats
            t=train_data['t'][:,i].astype('float32')
            x=train_data['x'][:,:,i].astype('float32')
            mu_0=np.concatenate((train_data['mu0'][:,i]).astype('float32')
            mu_1=np.concatenate((train_data['mu1'][:,i]).astype('float32')

            data={'x':x,'t':t,'y':y,'t':t,'mu_0':mu_0,'mu_1':mu_1}
            data['t']=data['t'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension 
            data['y']=data['y'].reshape(-1,1)

            #rescaling y between 0 and 1 often makes training of DL regressors easier
            data['y_scaler'] = StandardScaler().fit(data['y'])
            data['ys'] = data['y_scaler'].transform(data['y'])
    else:
           
        with open(testing_data,'rb') as tef:
            test_data=np.load(tef)
            y=  test_data['yf'][:,i].astype('float32') #most GPUs only compute 32-bit floats
            t= test_data['t'][:,i].astype('float32')
            x= test_data['x'][:,:,i].astype('float32')
            mu_0= test_data['mu0'][:,i].astype('float32')
            mu_1= test_data['mu1'][:,i].astype('float32')

            data={'x':x,'t':t,'y':y,'t':t,'mu_0':mu_0,'mu_1':mu_1}
            data['t']=data['t'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension 
            data['y']=data['y'].reshape(-1,1)

            #rescaling y between 0 and 1 often makes training of DL regressors easier
            data['y_scaler'] = StandardScaler().fit(data['y'])
            data['ys'] = data['y_scaler'].transform(data['y'])

    return data
 
def cal_pehe(i,model):

    data=load_IHDP_data_n('./ihdp_npci_1-100.train.npz','./ihdp_npci_1-100.test.npz',i,'test')

    concat_pred=model.predict(data['x'])
    #dont forget to rescale the outcome before estimation!
    y0_pred = data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
    y1_pred = data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))



    cate_pred=y1_pred-y0_pred
    cate_true=data['mu_1']-data['mu_0'] #Hill's noiseless true values
    #ate_pred=tf.reduce_mean(cate_pred)
    #print("Estimated ATE (True is 4):", ate_pred.numpy(),'\n\n')

    #print("Individualized CATE Estimates: BLUE")
    #print(pd.Series(cate_pred.squeeze()).plot.kde(color='blue'))

    #print("Individualized CATE True: Green")
    #print(pd.Series(cate_true.squeeze()).plot.kde(color='green'))

    #print("\nError CATE Estimates: RED")
    #print(pd.Series(cate_pred.squeeze()-cate_true.squeeze()).plot.kde(color='red'))


    cate_err=tf.reduce_mean( tf.square( ( (data['mu_1']-data['mu_0']) - (y1_pred-y0_pred) ) ) )

    return tf.math.sqrt(cate_err)
     

In [98]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD, Adam
pehe_loss=[]
val_split=0.2
batch_size=100
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)
 #we'll use both y and t to compute the loss
sgd_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=40, min_delta=0.), 
        #40 is Shi's recommendation for this dataset, but you should tune for your data 
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, verbose=verbose, mode='auto',
                          min_delta=0., cooldown=0, min_lr=0),
    ]
#optimzier hyperparameters
sgd_lr = 1e-5
momentum = 0.9
#tarnet_model.compile(optimizer=SGD(lr=sgd_lr, momentum=momentum, nesterov=True),
                    #loss=regression_loss,
                    #metrics=regression_loss)



for i in range(0,100):
   
    
    data=load_IHDP_data_n('./ihdp_npci_1-100.train.npz','./ihdp_npci_1-100.test.npz',i,'train')
    tarnet_model.compile(optimizer=Adam(learning_rate=1e-4),
                    loss=regression_loss,
                    metrics=regression_loss)
    
    yt = np.concatenate([data['ys'], data['t']], 1)

    tarnet_model.fit(x=data['x'],y=yt,
                    callbacks=sgd_callbacks,
                    validation_split=val_split,
                    epochs=300,
                    batch_size=batch_size,
                    verbose=verbose)
    
    pehe_loss.append(cal_pehe(i,tarnet_model))
    
print("DONE!")


In [100]:
print(np.mean(pehe_loss))